In [ ]:
pip install azure-ai-ml
pip install mlflow


In [ ]:
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

# Set up your Azure ML client
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential, 
    subscription_id="262a3c1d-7fe3-4bfb-9f90-7a5b89109336".
    resource_group_name="CoreInvest_2023-POC",
    workspace_name="SSExperiments"
)

# Register the model in Azure ML
mlflow_model_uri = "runs:/<run-id>/model"  # Example for MLflow model URI
ml_model = Model(name="my_model", path=mlflow_model_uri, description="MLflow model")

ml_client.models.create_or_update(ml_model)


In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Define endpoint details
endpoint_name = "mlflow-model-endpoint"
endpoint = ManagedOnlineEndpoint(name=endpoint_name)

# Create the endpoint
ml_client.online_endpoints.create_or_update(endpoint)

# Define deployment details
deployment_name = "mlflow-deployment"
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model="my_model:1",  # Replace with your model name and version
    instance_type="Standard_DS3_v2",  # Choose the appropriate instance type
    instance_count=1
)

# Create deployment
ml_client.online_deployments.create_or_update(deployment)

# Update endpoint traffic to route to this deployment
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.create_or_update(endpoint)


In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Set up the client and test the endpoint
ml_client = MLClient(
    credential=DefaultAzureCredential(), 
    subscription_id="your-subscription-id", 
    resource_group_name="your-resource-group", 
    workspace_name="your-ml-workspace"
)

# Define test input
test_data = {"data": [[1, 2, 3, 4]]}

# Send request to the endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name="mlflow-model-endpoint",
    request_body=test_data
)

print("Response: ", response)
